Fine-tuning best T5 Transformer 🤖
-----------------------------------

In this notebook, we will continue the fine-tuning of T5 transformer on the new extracted sentences from the bool **Grammaire de Wolof Moderne**. We obtained, after a hyperparameter tuning with `wandb`, a best bleu score of **2.47** for french to wolof translation model. We provide, bellow, the main evaluation figures, obtained from the hyperparameter search step.

- Parallel coordinates from panel:


`Parameter importance char` (from [panel]():

![parameter_importance]()

In [1]:
# let us extend the paths of the system
import sys
import os

# path = "/content/drive/MyDrive/Memoire/subject2/T5/"

# sys.path.extend([path, f"{path}new_data"])

In [2]:
os.environ["WANDB_DISABLED"] = "true"

In [3]:
# !pip install -qq wandb --upgrade

In [4]:
# !pip install evaluate -qq
# !pip install sacrebleu -qq
# !pip install optuna -qq
# !pip install transformers -qq 
# !pip install tokenizers -qq
# !pip install nlpaug -qq
# !pip install ray[tune] -qq
# !python -m spacy download fr_core_news_lg 

In [5]:
# let us import all necessary libraries
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5TokenizerFast, set_seed
from wolof_translate.utils.sent_transformers import TransformerSequences
from wolof_translate.data.dataset_v2 import T5SentenceDataset
from wolof_translate.utils.sent_corrections import *
from sklearn.model_selection import train_test_split
from nlpaug.augmenter import char as nac
from torch.utils.data import DataLoader
# from datasets  import load_metric # make pip install evaluate instead
# and pip install sacrebleu for instance
from functools import partial
from tqdm import tqdm
import pandas as pd
import numpy as np
import evaluate
import wandb
import torch



c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


--------------

## French to wolof

### Configure dataset 🔠

In [6]:
def split_data(random_state: int = 50):
  """Split data between train, validation and test sets

  Args:
    random_state (int): the seed of the splitting generator. Defaults to 50
  """
  # load the corpora and split into train and test sets
  corpora = pd.read_csv(f"data/additional_documents/diagne_sentences/extractions.csv")

  train_set, test_set = train_test_split(corpora, test_size=0.1, random_state=random_state)

  # let us save the final training set when performing

  train_set, valid_set = train_test_split(train_set, test_size=0.1, random_state=random_state)

  train_set.to_csv(f"data/additional_documents/diagne_sentences/final_train_set.csv", index=False)

  # let us save the sets
  train_set.to_csv(f"data/additional_documents/diagne_sentences/train_set.csv", index=False)

  valid_set.to_csv(f"data/additional_documents/diagne_sentences/valid_set.csv", index=False)

  test_set.to_csv(f"data/additional_documents/diagne_sentences/test_set.csv", index=False)

In [7]:
# recuperate the tokenizer from a json file
tokenizer = T5TokenizerFast(tokenizer_file=f"wolof-translate/wolof_translate/tokenizers/t5_tokenizers/tokenizer_v3.json")


In [8]:
def recuperate_datasets(fr_char_p: float, fr_word_p: float):

  # Create augmentation to add on French sentences
  fr_augmentation = TransformerSequences(nac.KeyboardAug(aug_char_p=fr_char_p, aug_word_p=fr_word_p),
                                        remove_mark_space, delete_guillemet_space)

  # Recuperate the train dataset
  train_dataset_aug = T5SentenceDataset(f"data/additional_documents/diagne_sentences/final_train_set.csv",
                                        tokenizer,
                                        truncation = True,
                                        cp1_transformer = fr_augmentation)

  # Recuperate the test dataset
  test_dataset = T5SentenceDataset(f"data/additional_documents/diagne_sentences/test_set.csv",
                                        tokenizer,
                                        truncation = True)
  
  # Return the datasets
  return train_dataset_aug, test_dataset

### Configure the model and the evaluation function ⚙️

Let us recuperate the model and resize the token embeddings.

In [9]:
def t5_model_init(tokenizer):

  # Initialize the model name
  model_name = 't5-small'
  # model_name = 'data/checkpoints/vf_t5_small_v2_checkpoints_2/' # from checkpoint

  # import the model with its pre-trained weights
  model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

  # resize the token embeddings
  model.resize_token_embeddings(len(tokenizer))

  return model

Let us evaluate the predictions with the `bleu` metric.

In [10]:
# %%writefile wolof-translate/wolof_translate/utils/evaluation.py
from tokenizers import Tokenizer
from typing import *
import numpy as np
import evaluate

class TranslationEvaluation:
    
    def __init__(self, 
                 tokenizer: Tokenizer,
                 decoder: Union[Callable, None] = None,
                 metric = evaluate.load('sacrebleu'),
                 ):
        
        self.tokenizer = tokenizer
        
        self.decoder = decoder
        
        self.metric = metric
    
    def postprocess_text(self, preds, labels):
        
        preds = [pred.strip() for pred in preds]
        
        labels = [[label.strip()] for label in labels]
        
        return preds, labels

    def compute_metrics(self, eval_preds):

        preds, labels = eval_preds

        if isinstance(preds, tuple):
        
            preds = preds[0]
        
        decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

        labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
        
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

        decoded_preds, decoded_labels = self.postprocess_text(decoded_preds, decoded_labels)

        result = self.metric.compute(predictions=decoded_preds, references=decoded_labels)
        
        result = {"bleu": result["score"]}

        prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
        
        result["gen_len"] = np.mean(prediction_lens)
        
        result = {k: round(v, 4) for k, v in result.items()}
        
        return result

In [11]:
# %run wolof-translate/wolof_translate/utils/evaluation.py

Let us initialize the evaluation object.

In [12]:
evaluation = TranslationEvaluation(tokenizer)


### Searching for the best parameters 🕖

Let us define the data collator.

In [13]:
def data_collator(batch):
    """Generate a batch of data to provide to trainer

    Args:
        batch (_type_): The batch

    Returns:
        dict: A dictionary containing the ids, the attention mask and the labels
    """
    input_ids = torch.stack([b[0].squeeze(0) for b in batch])
    
    attention_mask = torch.stack([b[1].squeeze(0) for b in batch])
    
    labels = torch.stack([b[2].squeeze(0) for b in batch])
    
    return {'input_ids': input_ids, 'attention_mask': attention_mask,
            'labels': labels}

Let us initialize the training arguments and make random search.

In [14]:
# %%wandb

"""Best parameters
learning_rate = 0.0029455426961160418
weight_decay = 0.3273145442978588
train_batch_size = 16
random_state = 2
fr_char_p = 0.2646960611549013
fr_word_p = 0.32759507689127154
eval/bleu = 3.0599
"""

# let us define a directory
directory = "data/checkpoints/t5_results_fw_v2_3"

# seed
set_seed(0)

# split the data
split_data(random_state=2)

# let us recuperate the datasets
train_dataset, test_dataset = recuperate_datasets(0.2646960611549013, 0.32759507689127154)

# set training arguments
training_args = Seq2SeqTrainingArguments(directory,
                                    logging_dir="data/logs/results_fw_v2_3",
                                    overwrite_output_dir=True,
                                    num_train_epochs=500,
                                    load_best_model_at_end=True,
                                    save_strategy="epoch",
                                    evaluation_strategy="epoch",
                                    logging_strategy="epoch",
                                    per_device_train_batch_size=16, 
                                    per_device_eval_batch_size=16,
                                    learning_rate=0.0029455426961160418,
                                    # learning_rate=0.00003113,
                                    weight_decay=0.3273145442978588,
                                    predict_with_generate=True, # we will use predict with generate in order to obtain more valuable test results
                                    fp16 = True,
                                    metric_for_best_model = 'bleu', # a bleu score will be used to find the best model
                                    greater_is_better = True,
                                    save_total_limit = 2, # we will save only the best model
                                    )   

# define training loop
trainer = Seq2SeqTrainer(model_init=partial(t5_model_init, tokenizer = train_dataset.tokenizer),
                  args=training_args,
                  train_dataset=train_dataset, 
                  eval_dataset=test_dataset,
                  data_collator=data_collator,
                  compute_metrics=evaluation.compute_metrics
                  )

# load last checkpoint
# trainer._load_from_checkpoint("data/training2/results/checkpoint-147")

# start training loop
trainer.train()
# trainer.train(resume_from_checkpoint=True)
# trainer.train('data/checkpoints/t5_results_fw_v2_2/')
# trainer.train('data/checkpoints/vf_t5_small_v2_checkpoints_2/')
# trainer.train('data/checkpoints/vf_t5_small_v2_checkpoints/') # from the searching best model
# trainer.train('data/checkpoints/results_fw_v2/last_checkpoint/') # from last checkpoint



Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
c:\Users\Oumar Kane\AppData\Local\pypoetry\Cache\virtualenvs\pytorch1-HleOW5am-py3.10\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 151/75500 [00:27<2:58:32,  7.03it/s]

{'loss': 0.6273, 'learning_rate': 0.0029396906245343544, 'epoch': 1.0}


                                                     
  0%|          | 151/75500 [00:33<2:58:32,  7.03it/s]

{'eval_loss': 0.5181533098220825, 'eval_bleu': 0.6952, 'eval_gen_len': 4.1582, 'eval_runtime': 6.2139, 'eval_samples_per_second': 47.796, 'eval_steps_per_second': 3.058, 'epoch': 1.0}


  0%|          | 302/75500 [00:58<2:52:12,  7.28it/s] 

...

                                                       
100%|██████████| 75500/75500 [5:42:55<00:00,  6.23it/s]

{'eval_loss': 0.4737825095653534, 'eval_bleu': 17.9489, 'eval_gen_len': 5.2391, 'eval_runtime': 6.9379, 'eval_samples_per_second': 42.808, 'eval_steps_per_second': 2.739, 'epoch': 500.0}


100%|██████████| 75500/75500 [5:42:56<00:00,  3.67it/s]

{'train_runtime': 20576.0209, 'train_samples_per_second': 58.393, 'train_steps_per_second': 3.669, 'train_loss': 0.07788922929763795, 'epoch': 500.0}


TrainOutput(global_step=75500, training_loss=0.07788922929763795, metrics={'train_runtime': 20576.0209, 'train_samples_per_second': 58.393, 'train_steps_per_second': 3.669, 'train_loss': 0.07788922929763795, 'epoch': 500.0})

In [16]:
trainer.state.best_model_checkpoint

'data/checkpoints/t5_results_fw_v2_3\\checkpoint-69460'

In [ ]:
# # let us save the best model
# trainer.save_state()
# trainer.save_model('data/checkpoints/t5_results_fw_v2_2/')
# trainer._save('data/checkpoints/')

# with open('data/checkpoints/t5_results_fw_v2_2/optimizer.pt', 'wb') as f:
    
#     torch.save(trainer.optimizer.state_dict(), f)
    
# with open('data/checkpoints/t5_results_fw_v2_2/scheduler.pt', 'wb') as f:
    
#     torch.save(trainer.lr_scheduler.state_dict(), f)


In [17]:
# let us get the best model
model = AutoModelForSeq2SeqLM.from_pretrained('data/checkpoints/t5_results_fw_v2_3/checkpoint-69460')

### Predictions

Let us generate texts and store into a DataFrame.

In [18]:

# set the model to eval mode
_ = model.eval()

# run model inference on all test data
original_translations, predicted_translations, original_texts, scores = [], [], [], {}

for data, attention_mask, labels in tqdm(DataLoader(test_dataset)):
    
    # Traduce the sentences
    original_text = tokenizer.decode(data[0], skip_special_tokens=True)
    
    original_translation = tokenizer.decode(labels[0], skip_special_tokens=True)
    
    # get tokens
    generated = torch.tensor(data)
    
    attention_mask = torch.tensor(attention_mask)
    
    # recuperate the pad token id
    pad_token_id = tokenizer.pad_token_id
    
    # perform prediction
    predictions = model.generate(generated, do_sample = False, top_k = 50, max_length = test_dataset.max_len, top_p = 0.90,
                                    temperature = 0, num_return_sequences = 0, attention_mask = attention_mask, pad_token_id = pad_token_id)
    
    # calculate the score and add it to the score
    result = evaluation.compute_metrics((predictions, torch.tensor(labels)))
    
    if not scores: scores.update({k: v for k, v in result.items()})
    
    else: scores.update({k: round(scores[k] + v, 4) for k, v in result.items()})
    
    # decode the predicted tokens into texts
    predicted_translation = list(test_dataset.decode(predictions))
    
    print(predicted_translation[0])
    
    # append results
    original_translations.append(original_translation)
    
    predicted_translations.extend(predicted_translation)
    
    original_texts.append(original_text)

# transform result into data frame
df_ft_to_wf = pd.DataFrame({'original_text': original_texts,
                            'original_label': original_translations,
                            'predicted_label': predicted_translations})

# print the result
df_ft_to_wf.head()

  0%|          | 0/297 [00:00<?, ?it/s]C:\Users\Oumar Kane\AppData\Local\Temp\ipykernel_28364\2073266963.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  generated = torch.tensor(data)
C:\Users\Oumar Kane\AppData\Local\Temp\ipykernel_28364\2073266963.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask)
C:\Users\Oumar Kane\AppData\Local\Temp\ipykernel_28364\2073266963.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  result = evaluation.compute_metrics((predictions, torch.tensor(labels))

leen


  1%|          | 2/297 [00:02<05:14,  1.07s/it]

dog


  1%|          | 3/297 [00:02<04:36,  1.06it/s]

...

100%|█████████▉| 296/297 [04:10<00:00,  1.13it/s]

Góor gi bëgg na


100%|██████████| 297/297 [04:11<00:00,  1.18it/s]

Man demuma


,original_text,original_label,predicted_label
0,Va les voir!,Gisi leen!,leen
1,couper,dagg,dog
2,Ce n'était pas un homme de Saint-Louis.,Du woon góoru Ndar.,Du góor.
3,Peut-être l'homme a-t-il dit que c'est celui-là!,Soo demee góor gee ni kookule la!,Soo demee góor gee ni nit la
4,indignité,goreedi,waaraw


In [19]:
df_ft_to_wf.tail(10)

,original_text,original_label,predicted_label
287,fermer,up,ub
288,de quelle manière,naan,nule
289,C'est peut-être un lion!,"Ku dem, gaynde la!","Gaynde la, ku dem"
290,suinter,siit,pac
291,"Autres, ils sont.",Ñeñeen lañu.,Ñeneen lañu.
292,il y a un moment,saõx,gëmm nitu xam
293,qui nie,weddikat,waxkat
294,"Tu sais, cet homme?",Gis õga nit kookale?,Waaye nit kookii?
295,L'homme partira aujourd'hui,Góor gi dana dem tay ji,Góor gi bëgg na
296,Moi-même je n'ai pas été,Man mii demuma,Man demuma


In [20]:
# let us display 100 samples
pd.options.display.max_rows = 100
df_ft_to_wf.sample(100)

,original_text,original_label,predicted_label
167,"Je veux, si vous avez fini, que tu viennes et ...",Bëgg naa õga ñëw mu dem su õgeen noppée!,"Bëgg naa góor ñi ñëw, xale yi ñëw, jigéen ñi t..."
211,celui-ci près de toi,kookii,foofale
63,Du moment qu'il part,Bi mu dee dem,Bi mu dee dem
154,là a cet endroit,foofile,foofa
5,Est-ce comme je le crains que...!,Mbaa...!,Mbaa...
77,êtres humains de sexe masculin,nit ñu góor,nit ku góor
183,On croyait que tu allais partir matin et soir!,Dañu defewoon ni daa dem ci subë ag ci õgoon y...,Dañu defewoon ci õgoon ag ci suba yépp da dem
158,traîner,wàtal,wàt
9,L'homme n'a rien mangé avec la main.,Nit ki lekkul dara ak loxoom.,Góor gi waxtaan na ag yaw.
139,L'homme a donné quelque chose à quelqu'un.,Góor gi may na dara nit.,Góor gi may na nit dara.


## Colab download and remove step

In [ ]:
import shutil

# shutil.rmtree('/content/drive/MyDrive/Memoire/subject2/training2/results2')
shutil.rmtree('wandb')
# shutil.make_archive('wandb', 'zip', 'wanbd')